# Endpoint Search

# File operations

### Path encoding & UTF gotchas

### List directory content

### Make directory

### Rename

# Task submission

### Transfer w/ and w/o sync

### Delete

# Task management

### Get task_list

### Get task by id

### Update task by id (label, deadline)

### Cancel task by id

### Get event list for task

### Get task pause info

# Bookmarks

### CRUD + list

# Shared endpoints

### Access_manager role

### Get list off access rules

### Get access rule by id

### Create access rule

### Update access rule

### Delete access rule